In [1]:
#from ib_insync import *
import pandas as pd
import datetime 
from functools import reduce
from os import listdir
from os.path import isfile, join
from collections import defaultdict

In [2]:
forex_path = 'Forex/'
currencies = ["AUD_USD_minute_", "USD_HKD_minute_", "USD_JPY_minute_"]
currency_times = [("19:00:00", "20:00:00"), ("20:30:00", "21:30:00"), ("19:00:00", "20:00:00")]
new_csv_names = ["AUD_USD_new.csv", "USD_HKD_new.csv", "USD_JPY_new.csv"]

In [3]:
for i in range(3):
    # Collect all data in one df
    filenames = sorted([f for f in listdir(forex_path) if currencies[i] in f])
    df = pd.DataFrame(columns = ["date", "avg_bid", "avg_ask"])
    for filename in filenames:
        new_df = pd.read_csv(forex_path + filename, index_col = 0).rename(columns = {'close':'avg_ask', 'open':'avg_bid'})[["date", 'avg_bid', "avg_ask"]]
        df = pd.concat([df, new_df])
        
    # Filter for correct data
    df = df[(df["date"] > "2015-04")] 
    df = df.drop_duplicates()
    df = df.reset_index(drop = True)
    
    # Filter into US/Non-US
    us_df = df[df["date"].str.endswith("09:30:00") | df["date"].str.endswith("09:29:00")].copy()
    indices = df["date"].str.endswith(currency_times[i][0]) | df["date"].str.endswith(currency_times[i][1])
    prev_indices = indices.shift(-1)
    prev_indices[len(prev_indices)-1] = False
    non_us_df = df[indices | prev_indices].copy()
    print("We have {} rows of US market open data".format(len(us_df)))
    print("We have {} rows of non-US market open data".format(len(non_us_df)))
    
    # Convert Asian market opens into local date
    for j in range(len(non_us_df)):
        if j % 2 == 1:
            date_str = non_us_df.iloc[j,0]
            day_str, time_str = tuple(date_str.split())
            day = datetime.datetime.strptime(day_str, '%Y-%m-%d')
            day += datetime.timedelta(days=1)
            new_day = day.strftime('%Y-%m-%d')
            non_us_df.iloc[j,0] = new_day + " " + time_str
            date_str = non_us_df.iloc[j-1,0]
            day_str, time_str = tuple(date_str.split())
            non_us_df.iloc[j-1,0] = new_day + " " + time_str
            
    # Split datasets, merge into one, save
    us_df_before = us_df[us_df["date"].str.endswith("9:00")].copy()
    us_df_at = us_df[us_df["date"].str.endswith("0:00")].copy()
    # Check
    print("There are {} and {} rows of US market data before and at market open respectively.".format(len(us_df_before), len(us_df_at)))

    non_us_df_before = non_us_df[non_us_df["date"].str.endswith("9:00")].copy()
    non_us_df_at = non_us_df[non_us_df["date"].str.endswith("0:00")].copy()
    # Check
    print("There are {} and {} rows of non-US market data before and at market open respectively.".format(len(non_us_df_before), len(non_us_df_at)))

    non_us_df_before.rename(columns = {'avg_bid':'avg_bid_non_us_before', 'avg_ask':'avg_ask_non_us_before'}, inplace = True)
    non_us_df_before["date"] = non_us_df_before["date"].str.split().str[0]
    non_us_df_at.rename(columns = {'avg_bid':'avg_bid_non_us_at', 'avg_ask':'avg_ask_non_us_at'}, inplace = True)
    non_us_df_at["date"] = non_us_df_at["date"].str.split().str[0]
    us_df_before.rename(columns = {'avg_bid':'avg_bid_us_before', 'avg_ask':'avg_ask_us_before'}, inplace = True)
    us_df_before["date"] = us_df_before["date"].str.split().str[0]
    us_df_at.rename(columns = {'avg_bid':'avg_bid_us_at', 'avg_ask':'avg_ask_us_at'}, inplace = True)
    us_df_at["date"] = us_df_at["date"].str.split().str[0]
    final_df = reduce(lambda x, y : pd.merge(x, y), [non_us_df_before, non_us_df_at, us_df_before, us_df_at])
    final_df.to_csv(forex_path + new_csv_names[i])

We have 3134 rows of US market open data
We have 3130 rows of non-US market open data
There are 1567 and 1567 rows of US market data before and at market open respectively.
There are 1565 and 1565 rows of non-US market data before and at market open respectively.
We have 3134 rows of US market open data
We have 3128 rows of non-US market open data
There are 1567 and 1567 rows of US market data before and at market open respectively.
There are 1564 and 1564 rows of non-US market data before and at market open respectively.
We have 3130 rows of US market open data
We have 3128 rows of non-US market open data
There are 1565 and 1565 rows of US market data before and at market open respectively.
There are 1564 and 1564 rows of non-US market data before and at market open respectively.
